In [27]:
import numpy as np
import os
import cv2
from tensorflow.keras import applications, models, layers

In [28]:
# 데이터 전처리
def get_data(path):
    x = []
    y = []
    dir_list = os.listdir(path)
    for i in range(len(dir_list)-1):
        dir_path = path + "/" + dir_list[i+1]
        dir_name = os.listdir(dir_path)

        for j in range(len(dir_name)):
            full_dir_path = dir_path + "/" + dir_name[j]

            img = cv2.imread(full_dir_path)
            img = cv2.resize(img, (32,32))
            x.append(img)
            y.append(i)
    # print(x)
    # print(y)
    return np.array(x),np.array(y)
#get_data()

In [29]:
# 모델 생성
def make_model() :
    resnet50 = applications.resnet50.ResNet50(include_top=False, weights="imagenet", input_shape=(700,500,3))
    resnet50.trainable = False
    model = models.Sequential()
    model.add(resnet50)
    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation="relu"))
    model.add(layers.Dense(512, activation="relu"))

    # 분류 데이터가 몇종류인지
    model.add(layers.Dense(2, activation="softmax"))
    model.compile(loss = "sparse_categorical_crossentropy", optimizer="adam", metrics = ["accuracy"])
    print(model.summary())
    return model

In [30]:
# 한개 테스트용
def get_test():
    test_path = "./test/non/test17.png"
    test_img = cv2.imread(test_path)
    #print(test_img)
    test_img = cv2.resize(test_img, (32,32))
    #test_img = test_img.reshape(32,32,3)
    test_list = []
    test_list.append(test_img)
    test_list = np.array(test_list)
    #print(test_img.shape)
    cv2.imwrite("test123.png",test_img)
    return test_list

In [33]:
# 모델 학습
def learning() :
    train_path = "./data_image"
    test_path = "./test"
    (x_train, y_train)= process_data(train_path)
    (x_test, y_test) = process_data(test_path)
    model = make_model()
    # print("x_train : ",x_train[0])
    # print("x_train.shape : ",x_train[0].shape)
    log = model.fit(x_train, y_train, epochs=2, batch_size=16)
    #model.save("face.h5")
    
    model.evaluate(x_test,y_test)
learning()

./data_image/non/1019.png
./data_image/non/1031.png
./data_image/non/1025.png
./data_image/non/1024.png
./data_image/non/1030.png
./data_image/non/1018.png
./data_image/non/1026.png
./data_image/non/1032.png
./data_image/non/1033.png
./data_image/non/1027.png
./data_image/non/1023.png
./data_image/non/1037.png
./data_image/non/1036.png
./data_image/non/1022.png
./data_image/non/1034.png
./data_image/non/1020.png
./data_image/non/1008.png
./data_image/non/1009.png
./data_image/non/1021.png
./data_image/non/1035.png
./data_image/non/1091.png
./data_image/non/1085.png
./data_image/non/1052.png
./data_image/non/1046.png
./data_image/non/1047.png
./data_image/non/1053.png
./data_image/non/1084.png
./data_image/non/1090.png
./data_image/non/1086.png
./data_image/non/1092.png
./data_image/non/1079.png
./data_image/non/1045.png
./data_image/non/1051.png
./data_image/non/1050.png
./data_image/non/1044.png
./data_image/non/1078.png
./data_image/non/1093.png
./data_image/non/1087.png
./data_image

ValueError: in user code:

    File "/Users/hongseongmin/miniforge3/envs/py39/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/hongseongmin/miniforge3/envs/py39/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/hongseongmin/miniforge3/envs/py39/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/hongseongmin/miniforge3/envs/py39/lib/python3.9/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/Users/hongseongmin/miniforge3/envs/py39/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/hongseongmin/miniforge3/env

In [31]:
import cv2
import numpy as np
path = "./tracking/test1.png"

img = cv2.imread(path, cv2.IMREAD_COLOR)

# 색 변경
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

img_hand = cv2.inRange(img_gray,(10), (165))
mask = np.zeros(img.shape).astype(img.dtype)

#경계선 찾음
contours, hierarchy = cv2.findContours(img_hand, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# 가장 큰 영역 찾기
max = 0
maxcnt = None
for cnt in contours :
    area = cv2.contourArea(cnt)
    if(max < area) :
        max = area
        maxcnt = cnt


# 경계선 내부 255로 채우기
color = [255, 255, 255]
cv2.fillPoly(mask, [maxcnt], color)
img_hand = cv2.bitwise_and(img, mask)
hull = cv2.convexHull(maxcnt)
#print(hull)

# 도출된 값으로 사각형 그리기
x,y,w,h = cv2.boundingRect(hull)
cv2.rectangle(mask, (x,y), (x+w,y+h), (0,255,0),3 )


cv2.imshow('image', mask)
#cv2.imshow('image', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)


-1

In [34]:
def process_data(path):
    x = []
    y = []
    dir_list = os.listdir(path)
    #print(os.listdir(path))
    for i in range(0,len(dir_list)-1):
        # if i == 1:
        #     break
        dir_path = path + "/" + dir_list[i+1]
        #print(os.listdir(dir_path))
        dir_name = os.listdir(dir_path)
        
        for j in range(len(dir_name)):
            
            full_dir_path = dir_path + "/" + dir_name[j]
            print(full_dir_path)
            img = cv2.imread(full_dir_path,cv2.IMREAD_COLOR)
            # 색 변경
            img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

            img_hsv = cv2.fastNlMeansDenoisingColored(img_hsv,None,10,10,7,21)
            lower = np.array([0,48,80], dtype="uint8")
            upper = np.array([20,255,255], dtype="uint8")
            img_hand = cv2.inRange(img_hsv,lower,upper)
            

            #경계선 찾음
            contours, hierarchy = cv2.findContours(img_hand, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            # 가장 큰 영역 찾기
            max = 0
            maxcnt = None
            for cnt in contours :
                area = cv2.contourArea(cnt)
                if(max < area) :
                    max = area
                    maxcnt = cnt
                    if maxcnt.all() == None:
                        break
            #print(maxcnt)
            
            mask = np.zeros(img.shape).astype(img.dtype)
            # 경계선 내부 255로 채우기
            color = [255, 255, 255]
            cv2.fillPoly(mask, [maxcnt], color)
            img_hand = cv2.bitwise_and(img, mask)
            
            
            mask = cv2.resize(mask, (700,500))
            cv2.imwrite('image.png', mask)
            x.append(mask)
            y.append(i)
    print(x.shape)
    print(y.shape)
    return np.array(x),np.array(y)

process_data("./data_image")


./data_image/non/1019.png
./data_image/non/1031.png
./data_image/non/1025.png
./data_image/non/1024.png
./data_image/non/1030.png
./data_image/non/1018.png
./data_image/non/1026.png
./data_image/non/1032.png
./data_image/non/1033.png
./data_image/non/1027.png
./data_image/non/1023.png
./data_image/non/1037.png
./data_image/non/1036.png
./data_image/non/1022.png
./data_image/non/1034.png
./data_image/non/1020.png
./data_image/non/1008.png
./data_image/non/1009.png
./data_image/non/1021.png
./data_image/non/1035.png


KeyboardInterrupt: 